# LangChain으로 만드는 웹 기반 챗봇 실습  

##  1. 과정 소개 및 개요
- **학습 목표**
  - AI 챗봇의 기본 개념과 발전 과정을 이해한다.
  - 규칙 기반 챗봇과 LLM 기반 챗봇을 비교할 수 있다.
  - LangChain, Streamlit, FAISS 등 주요 기술의 역할과 장점을 이해한다.
  - LCEL을 사용하여 확장성 있는 체인을 직접 설계할 수 있다.
  - **웹 기반으로 자신이 커스텀한 챗봇을 배포할 수 있는 역량을 갖춘다.**

- **핵심 키워드**
  - LLM Chatbot, LangChain, LCEL, Prompt, Streamlit, FAISS, RAG  

### 환경 설정 

#### (1) 필수 라이브러리 설치 
웹 페이지(UI) 를 직접 띄워야 하기 때문에, Google Colab보다는  
본인 PC의 로컬 환경(VSCode, Terminal 등) 에서 실행하는 것을 권장합니다.

먼저 실습에 필요한 주요 패키지를 설치합니다.
터미널 또는 VSCode의 터미널 창에서 다음 명령어를 입력하세요.
```bash
pip install -U langchain langchain-openai streamlit faiss-cpu python-dotenv tiktoken
```


#### (2) 라이브러리별 역할 설명

| 라이브러리                | 주요 역할        | 설명                                                                                                  |
| -------------------- | ------------ | --------------------------------------------------------------------------------------------------- |
| **langchain**        | 체인 프레임워크 핵심  | LLM(대형언어모델)을 중심으로 다양한 입력·출력·도구를 연결하는 프레임워크. `PromptTemplate`, `LLMChain`, `Retriever`, `LCEL` 등을 제공 |
| **langchain-openai** | OpenAI 연동 모듈 | OpenAI의 GPT 모델(`ChatOpenAI`)을 LangChain 체인 안에서 바로 사용할 수 있도록 연결해줌                                    |
| **streamlit**        | 웹 인터페이스 구현   | Python 코드 몇 줄로 대화형 웹 UI(챗봇 화면, 입력창, 출력창 등)를 만들 수 있는 간단한 프레임워크                                       |
| **faiss-cpu**        | 벡터 검색 DB     | 문서 검색 기반 챗봇(RAG) 구현 시, 텍스트 임베딩을 벡터로 저장하고 유사한 내용을 빠르게 검색하는 역할                                        |
| **python-dotenv**    | 환경변수 관리      | `.env` 파일에서 OpenAI API Key 등 민감한 정보를 안전하게 불러와 코드에 직접 노출하지 않게 해줌                                     |

#### (3) OpenAI API 키 설정 (.env 파일, 보안 주의)

**API란?**

하나의 프로그램이 다른 프로그램이나 시스템과 **데이터를 주고받고 기능을 활용할 수 있도록 연결해주는 인터페이스**를 의미합니다.  
즉, **서로 다른 소프트웨어 간의 다리 역할을 하는 통신 규칙**입니다.

예를 들어, OpenAI의 **GPT API**는 사용자가 직접 모델을 설치하지 않아도  
OpenAI 서버에 **질문(요청)** 을 보내고, 그 결과로 **답변(응답)** 을 받을 수 있게 해주는 API입니다.

→ 쉽게 말해, **필요할 때마다 GPT에게 요청을 보내 답변을 받아오는 온라인 시스템**이라고 생각하시면 됩니다.

※ 이 API는 **사용량 기반으로 과금되는 유료 서비스**이며, 웹에서 제공되는  
ChatGPT의 무료·유료 플랜과는 **별도의 요금 체계**로 운영됩니다.


※ 현재 사용 가능한 대표 GPT API 모델 일부 (2025년 10월 기준)

| 모델 이름 | 설명 | 입력 가격 (per 1M tokens) | 출력 가격 (per 1M tokens) |
|-----------|--------|---------------------------|----------------------------|
| `gpt-3.5-turbo` | 초창기 모델. 현재 더 저렴하고 성능이 좋은 gpt-4.1-mini 모델 등이 사용 가능| **\$0.5** | **\$1.5** |
| `gpt-4.1-mini` | 성능, 속도, 비용 측면에서 골고루 균형이 잡힌 모델| **\$0.4** | **\$1.6** |
| `gpt-4.1-nano` | 성능을 조금 희생하고 속도와 비용 측면에서 최적화를 이룬 모델| **\$0.1** | **\$0.4** |
| `o4-mini` | 빠르고 효과적인 추론에 최적화되어 있으며, 코딩 및 시각 작업에서 탁월한 성능을 발휘 | **\$1.1** | **\$4.4** |
| `gpt-5-mini` | 빠르고 효율적인 비용이며 명확한 작업을 수행하는 모델 | **\$0.25** | **\$2.0** |
| `gpt-5-nano` | GPT-5 모델을 빠르고 가장 효율적인 비용으로 활용할 수 있는 모델 | **\$0.05** | **\$0.4** |

※ OpenAI는 지속적으로 모델명을 업데이트하고 있으므로, [공식 문서](https://platform.openai.com/docs/models)에서 최신 목록을 확인하는 것이 좋습니다.

최신 요금은 OpenAI 공식 문서에서 확인하세요:  
https://openai.com/api/pricing/

**API Key란?**  
GPT API를 사용하기 위해서는 OpenAI 서버가 요청을 보낸 사용자를 식별하고,  
해당 사용자의 **이용량과 과금 내역을 추적**할 수 있어야 합니다.  
이를 위해 OpenAI는 각 사용자에게 **고유한 인증 토큰(API Key)** 을 발급합니다.

이 키는 **요금 청구의 기준이 되는 중요 정보**이며, 외부에 노출될 경우 **타인이 임의로  
사용해 요금이 발생할 수 있으므로 철저한 보안 관리가 필요**합니다.


**발급 절차**

1. [https://platform.openai.com/](https://platform.openai.com/) 접속  
2. 로그인 또는 계정 신규 생성  
3. 우측 상단 프로필 아이콘 클릭 → `View API Keys` 선택  
4. 결제 수단(신용카드 등) 등록  
5. `+ Create new secret key` 클릭 → **발급된 Key를 복사해 안전한 위치에 보관**

**발급된 Key는 한 번만 표시되므로**, 반드시 즉시 메모장이나 `.env` 파일 등에 백업해 두세요.

실습 폴더 루트에 .env 파일을 생성하고, 강사가 안내한 OpenAI API Key를 아래 형식으로 저장합니다. 
```bash
OPENAI_API_KEY=sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
``` 

#### ⚠️ 주의
.env 파일은 절대 깃허브(GitHub)나 외부 저장소에 업로드하지 마세요.  
사용은 `.env` 파일에 저장 후 `python-dotenv`를 이용해 불러오는 방식을 권장합니다.
API Key가 노출되면 과금 위험이 있습니다.

```python
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
```

##  2. LangChain 소개
### 🔹 LangChain이란?

<img src ="image/LangChain.png" width="500">

LangChain은 **대규모 언어모델(LLM)** 을 더 쉽게 활용할 수 있도록 도와주는 **프레임워크**입니다.  

단순히 LLM에게 질문을 던지는 수준을 넘어서, **프롬프트 관리(prompt management)**,  
**메모리(memory)**, **외부 도구(tool) 연동**, **문서 검색(RAG)** 등을 체계적으로 구성할 수 있습니다.

예를 들어, 우리가 ChatGPT 같은 LLM을 사용할 때  
단순히 모델 하나만으로 뛰어난 답변이 만들어지는 것은 아닙니다.  

실제로는 **검색(Search)**, **분석(Analysis)**, **정보 정리(Summarization)** 등  
여러 절차가 백그라운드에서 함께 작동해야 더 정확하고 풍부한 답변을 얻을 수 있습니다.  

LangChain은 이러한 여러 단계를 **하나의 체계적인 구조로 묶어주는 프레임워크**입니다.  
즉, LLM이 단독으로 모든 일을 처리하는 대신  
“사용자 입력 → 관련 정보 검색 → 분석 및 요약 → 결과 생성”과 같은  
복잡한 흐름을 하나의 **체인(chain)** 으로 자동화할 수 있습니다.  

<img src="image/chatgpt.jpg" width="600">

예를 들어, 사용자가 “3박 4일, 30만 원 예산, 맛집 중심의 서울 여행”을 요청하면,  
LangChain은 내부적으로 다음과 같은 과정을 수행합니다.  

> 입력 분석 → 예산과 일정 파악 → 추천 장소 검색 → 일정표 생성 → 자연스러운 문장으로 요약  

이처럼 LangChain은 단순한 대화 요청을 넘어 **검색, 분석, 조합, 요약** 등의 단계를 유기적으로 연결해  
LLM이 더 똑똑하게 작동하도록 돕습니다.  

또한 LangChain은 **LCEL (LangChain Expression Language)** 이라는 표현 언어를 통해  
이러한 여러 단계(components)를 **직관적으로 연결**할 수 있게 해줍니다.  
즉, “LLM → 요약기 → 출력 포맷터”처럼 **함수형 파이프라인**을 선언적으로 작성할 수 있어  
확장성과 유지보수성이 뛰어납니다.

LCEL 예시: 
```text
사용자입력
  | 분석기(요청에서 날짜·예산·키워드 추출)
  | 검색기(분석 결과를 바탕으로 추천 장소 탐색)
  | 일정생성기(추천 장소로 일자별 일정 구성)
  | 요약기(자연스러운 문장으로 결과 정리)
  | 출력
```

이처럼 LCEL은 각 단계를 함수처럼 “연결(pipe)”하여 데이터가 흘러가는 경로를 선언적으로 표현합니다.  
복잡한 제어문 없이 “입력 → 처리 → 출력” 구조를 직관적으로 구성할 수 있어,
체인 전체의 논리 흐름을 한눈에 파악할 수 있습니다.

In [1]:
# 1단계 | 2단계 | 3단계 | 4단게

### 🔹 간단한 예시 코드

In [2]:
# OpenAI API를 LangChain에서 쉽게 쓸 수 있도록 해주는 래퍼 모듈
from langchain_openai import ChatOpenAI
from langchain_core.prompts  import PromptTemplate # 문자열 템플릿 기반으로 프롬프트를 자동 구성해주는 유틸
from langchain_core.output_parsers import StrOutputParser # LLM의 응답 객체를 문자열로 변환해주는 파서 (출력 후처리 단계)
from dotenv import load_dotenv # .env 파일의 환경변수를 자동으로 불러오기 위한 모듈
load_dotenv()  # 실행 시 .env 파일을 찾아 변수들을 환경에 로드

# LCEL 방식: 파이프라인 형태로 선언
llm = ChatOpenAI(model="gpt-4o-mini")  # 성능은 낮지만 빠른 결과
prompt = PromptTemplate.from_template("'{topic}' 주제에 대해 한 문장으로 설명해줘.")
output_str = StrOutputParser() # 결과 객체에서 텍스트(content)만 깔끔하게 추출해 문자열로 변환함

# LCEL 표현 (프롬프트 → LLM → 출력)  자동화 파이프 라인 chain
chain = prompt | llm | output_str

# chain.invoke() 는 전체 파이프라인을 한 번 실행하는 메서드
result = chain.invoke({"topic": "LangChain"})
print(result)

LangChain은 다양한 언어 모델을 활용하여 복잡한 애플리케이션을 구축할 수 있도록 지원하는 프레임워크입니다.


| 항목            | 설명                              |
| ------------- | ------------------------------- |
| **LangChain** | LLM을 활용하기 위한 파이썬 기반 프레임워크       |
| **핵심 기능**     | 프롬프트 관리, 메모리, 도구 연동, 문서 검색(RAG) |
| **LCEL**      | 체인 구성 과정을 간결하게 표현하는 언어          |
| **활용 예시**     | 질문응답, 요약, 문서검색, 챗봇, 자동화된 지식시스템  |


LangChain은 LLM의 “두뇌”를 실무에서 “자동화 가능한 시스템”으로 바꿔주는 연결 프레임워크입니다.  
단순한 모델 호출이 아니라, 프로세스를 설계하고 실행하는 엔진이라고 이해하면 됩니다.

## 3. 프롬프트(Prompt)

**프롬프트(Prompt)** 는 LLM(대형언어모델)에게 “무엇을, 어떻게 해달라”고 **지시하는 문장**입니다.

사람에게 질문할 때도 요청을 명확히 해야 원하는 답을 얻을 수 있듯,  LLM에게도 프롬프트가 곧 **“명령어이자 대화의 시작점”** 이 됩니다.  

예를 들어,  
> 1. “고양이에 대해 설명해줘.”  
> 2. “고양이를 5살 아이가 이해할 수 있게 설명해줘.”  

두 문장은 같은 주제지만 결과는 완전히 다릅니다.  
👉 따라서 **프롬프트의 설계(=프롬프트 엔지니어링)** 가 답변의 질을 결정합니다.

#### LLM은 세 가지 종류의 프롬프트로 대화한다
LLM은 단순히 한 문장만 보고 답하지 않습니다.  
대화의 **맥락(context)** 을 유지하기 위해 다음 세 가지 프롬프트를 함께 사용합니다.

| 종류 | 역할 | 예시 |
|------|------|------|
| **System Prompt** | 모델의 성격과 역할을 설정 | “너는 친절하고 이해심 많은 선생님이야.” |
| **User Prompt** | 사용자의 실제 질문이나 요청 | “인공지능이 뭐야?” |
| **Assistant Prompt** | 모델이 이전에 했던 응답  ex) 예시 | “인공지능은 사람의 학습 능력을 모방한 기술이야.” |

#### 왜 이렇게 구분할까?
- **System Prompt** → 모델의 **기본 성격**을 결정합니다. (친절한 선생님 / 냉정한 분석가 등)  
- **User Prompt** → 사용자가 **무엇을 알고 싶은지**를 알려줍니다.  
- **Assistant Prompt** → 이전 대화 내용을 **기억하고 맥락을 유지**하게 도와줍니다.  

이 세 가지를 함께 쓰면 모델은  
> “지금 나는 친절한 선생님이고, 방금 ‘인공지능이 뭐냐’는 질문을 들었구나.”  
라고 이해한 뒤 더 자연스럽고 일관된 답변을 합니다.

### 🔹 LangChain에서의 프롬프트: `PromptTemplate`
LangChain에서는 프롬프트를 **하드코딩(직접 작성)** 하는 대신, **`PromptTemplate`** 클래스를 사용해 “템플릿” 형태로 관리합니다.

#### 예시 1

In [6]:
from langchain_core.prompts  import PromptTemplate

# {topic} 부분에 사용자의 입력이 들어가도록 템플릿 생성
prompt = PromptTemplate.from_template("'{topic}'에 대해 초보자도 이해할 수 있게 한 문장으로 설명해줘.")

# 실제 실행 시:
prompt.format(topic="LangChain")
# 결과 → "'LangChain'에 대해 초보자도 이해할 수 있게 한 문장으로 설명해줘."

"'LangChain'에 대해 초보자도 이해할 수 있게 한 문장으로 설명해줘."

#### 예시 2

In [20]:
parser

StrOutputParser()

In [19]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts  import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 1.  모델 선언
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

# 2️. 프롬프트 구성
# - System: 모델의 기본 역할(여행 전문가)
# - User: 사용자의 첫 질문 (가을 여행지 추천)
# - Assistant: 직전에 모델이 대답했던 내용 (AI의 응답)
# - User: 현재 사용자의 새로운 질문 (부산 일정표 요청)
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 친절한 여행 전문가야. 사용자의 취향에 맞게 국내 여행지를 추천하고, 일정도 구체적으로 제안해줘."),
    ("user", "가을에 2박 3일로 갈만한 국내 여행지를 추천해줘."),
    ("assistant", "부산, 여수, 속초가 좋을 것 같아요! 각각 바다와 음식, 분위기가 다르니 선택에 따라 즐길 거리가 달라요."),
    ("user", "그럼 이번엔 부산으로 2박 3일 일정표를 만들어줘.")
])

# 3️. 출력 파서 (응답 텍스트만 추출)
#parser = StrOutputParser()

# 4️. LCEL 파이프라인 구성 (Prompt → LLM → Parser)
chain = prompt | llm | parser

# 5️. 실행
result = chain.invoke({})

print(result)

부산으로의 2박 3일 여행 일정표를 아래와 같이 제안해드립니다.

### 1일차: 부산 도착 및 해운대 탐방
- **오전**
  - **부산 도착**: 기차나 비행기로 부산에 도착 후 숙소 체크인.
  - **점심식사**: 해운대 근처의 유명한 해물탕 식당에서 점심.

- **오후**
  - **해운대 해수욕장**: 해운대 해변에서 산책하고, 바다를 감상하세요.
  - **동백섬**: 동백섬을 산책하며 아름다운 경치를 감상하고, 누리마루 APEC 하우스 방문.

- **저녁**
  - **저녁식사**: 해운대 근처의 회센터에서 신선한 회와 해산물 요리 즐기기.
  - **해운대 야경**: 해운대 바닷가에서 야경 감상 후 숙소로 돌아가기.

### 2일차: 감천문화마을 & 자갈치시장
- **오전**
  - **아침식사**: 숙소에서 간단히 아침을 먹은 후 감천문화마을로 이동.
  - **감천문화마을 탐방**: 감천문화마을의 예쁜 벽화와 카페 탐방.

- **점심**
  - **점심식사**: 감천문화마을 근처의 전통 한식당에서 점심.

- **오후**
  - **자갈치시장 방문**: 자갈치시장에서 신선한 해산물 구경 및 간단한 간식(오징어튀김, 해물파전 등) 즐기기.
  - **BIFF 광장**: 부산국제영화제(BIFF) 광장에서 기념촬영 및 주변 카페 탐방.

- **저녁**
  - **부산 야경**: 광안리 해변에서 저녁식사 후 광안대교 야경 감상.

### 3일차: 태종대 & 송도 해수욕장
- **오전**
  - **아침식사**: 숙소에서 아침 후 태종대로 이동.
  - **태종대 탐방**: 태종대에서 자연경관 감상 및 유람선 탑승 가능.

- **점심**
  - **점심식사**: 태종대 인근의 해물 요리 전문점에서 신선한 해산물 점심.

- **오후**
  - **송도 해수욕장**: 송도 해수욕장에서 해변 산책 및 케이블카 탑승.
  - **부산 도심 탐방**: 부산 시내의 명소(부산타워, 용두산 공원 등) 방문.

- **저녁**
  - **저녁식사**:

In [18]:
result

AIMessage(content='부산에서의 2박 3일 여행 일정을 아래와 같이 제안해 드릴게요!\n\n### 1일차: 부산 도착 및 해운대 탐방\n- **오전**\n  - 부산 도착 후 숙소 체크인\n  - 해운대 해수욕장 산책 (해변에서 바다의 경치 감상)\n\n- **점심**\n  - 해운대 근처의 ‘해운대 암소갈비’에서 갈비찜 또는 소고기 요리 시식\n\n- **오후**\n  - 동백섬 탐방 (동백섬의 아름다운 자연과 전망대에서 해운대 전경 감상)\n  - APEC 나루공원 방문 (산책과 사진 촬영)\n\n- **저녁**\n  - 해운대 근처의 ‘해운대 시장’에서 다양한 길거리 음식 탐방 (오징어튀김, 떡볶이 등)\n\n- **밤**\n  - 해운대 바닷가에서 야경 감상 및 카페에서 차 한 잔\n\n### 2일차: 부산 문화 탐방\n- **오전**\n  - 감천문화마을 방문 (예쁜 벽화와 아기자기한 집들 탐방)\n  - 근처 카페에서 브런치\n\n- **점심**\n  - 광안리 해변 근처의 ‘광안리 횟집’에서 신선한 회 시식\n\n- **오후**\n  - 광안리 해변에서 해변 산책 및 사진 촬영\n  - 이기대 공원 트레킹 (바다를 바라보며 자연을 만끽)\n\n- **저녁**\n  - 부산의 명물 ‘부산어묵’ 시식 후, 서면에서 저녁식사 (추천: 부산의 다양한 전통 음식)\n\n- **밤**\n  - 서면의 번화가 탐방 및 쇼핑 (기념품 구매)\n\n### 3일차: 역사와 자연 탐방\n- **오전**\n  - 부산타워 방문 (부산 전경 감상)\n  - 용두산 공원 산책\n\n- **점심**\n  - 자갈치 시장에서 신선한 해산물 점심 (회, 조개구이 등)\n\n- **오후**\n  - 태종대 방문 (태종대 등대와 해안 절경 감상)\n  - 유람선 투어 (바다에서 태종대의 절경 감상)\n\n- **저녁**\n  - 여행 마무리로 부산의 유명한 ‘밀면’ 또는 ‘국수’ 시식\n\n- **밤**\n  - 귀가 준비 후, 부산역으로 이동\n\n이 일정은 부산의 

#### 예시 3. 퓨샷 러닝(Few-shot Learning)

**Few-shot Learning**은 LLM에게 **“예시 몇 개를 직접 보여줘서 학습 방향을 잡아주는 방법”** 입니다.

AI에게 “이렇게 대답해!”라고 명령하는 것만으로는 부족할 때,  
실제 **입출력 예시를 몇 개 제시해주면** 모델이 그 **패턴과 말투, 구조**를 빠르게 학습해 유사한 답변을 생성합니다.

예를 들어  
> “백종원 대표의 충청도 사투리로 요리 팁을 말해줘.”  
라고만 하면 AI는 사투리의 뉘앙스를 정확히 몰라 평범한 답변을 할 수 있습니다.  

하지만 아래처럼 실제 대화 예시를 1~2개 보여주면  
> “아~ 이런 식으로 말하면 되는구나!”  
하고 패턴을 스스로 학습해 자연스러운 말투를 재현합니다.

**퓨샷 러닝의 장점**

| 장점 | 설명 |
|------|------|
|**말투·스타일 학습 가능** | 특정 인물(예: 백종원, 아이유, 선생님 등)의 말투나 표현방식 재현 |
|**작업 포맷 유지** | 요약, 번역, 해설 등 특정 형식을 예시로 제시하면 그 구조를 그대로 따름 |
|**추가 학습(파인튜닝) 없이 가능** | 단순히 프롬프트 안에 예시 몇 개 넣는 것만으로도 “즉석 학습” 효과 |
|**창의적 제어** | “~처럼 말해줘”, “이런 형식으로 답해줘” 등 자유로운 응용 가능 |


In [23]:
### 코드 예시: 백종원 말투 흉내내기 (Few-shot Learning)
from langchain_openai import ChatOpenAI
from langchain_core.prompts  import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# temperature를 약간 높이면(0.8) 표현이 더 자유롭고 창의적이게 됨
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.8)

# - System: 모델의 기본 역할 지정
# - 예시 대화(few-shot examples): 실제 말투 패턴을 보여줌
# - User: 새로운 질문
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 요리 전문가이자, 백종원 대표의 말투를 자연스럽게 흉내내는 AI야. 충청도 사투리와 친근한 말투로 대답해."),
    
    # [예시 1] — 사용자가 물어봤고, 백종원 말투로 답변한 사례
    ("user", "라면 맛있게 끓이는 비법 알려주세요."),
    ("assistant", "그라믄 말이여~ 라면은 물 끓일 때 스프를 같이 넣는 게 포인트여. 그래야 국물이 진~해진다잉."),
    
    # [예시 2] — 또 다른 백종원식 대답 예시
    ("user", "김치찌개 맛있게 끓이는 법은요?"),
    ("assistant", "김치찌개는 묵은지 써야혀~ 그래야 깊은 맛이 나. 돼지고기 좀 넣어주면 금상첨화여잉."),

    # [예시 3] — 또 다른 백종원식 대답 예시
    ("user", "집에 버터가 없는데 파스타 만들어도 될까?"),
    ("assistant", "아 그럼유, 없어도 돼유. 버터 없다고 파스타 못 만드는 거 아니유."),

    # [예시 4] — 또 다른 백종원식 대답 예시
    ("user", "요리를 너무 못해서 항상 실패해"),
    ("assistant", "처음엔 레시피 그대로 따라 해봐유.조금씩 해보다 보면 손에 익어유."),
    
    # [새로운 요청] — 실행 시 전달될 질문 변수. 이제 모델이 같은 말투로 이어서 대답해야 함
    ("user", "{question}")])

# 출력 파서 (응답 텍스트만 추출)
parser = StrOutputParser()

# LCEL 파이프라인 구성 (Prompt → LLM → Parser)
chain = prompt | llm | parser

# 실행 — invoke()에 직접 새로운 질문 전달
result = chain.invoke({"question": "내가 요리를 잘 할 수 있을까요? 어떻게 만들면 맛있어요?"})

print(result)

물론이지유! 요리는 연습이여. 처음엔 간단한 것부터 시작해보구, 레시피 보고 천천히 해보면 좋아유. 매일매일 조금씩 해보면 차츰 늘게 되어있어. 그리고 맛보면서 조절하는 것도 잊지 마유!


##  4. Q&A 챗봇 실습 (LCEL)

앞서 배운 **프롬프트 설계**와 **LCEL 파이프라인 구성법**을 직접 활용하여  
“질문 → 답변” 형태의 **Q&A 챗봇**을 완성해보세요.

단순히 모델을 호출하는 것이 아니라, **프롬프트 구조를 설계**하고 **출력 파서를 연결**하며  
 **LCEL 문법(| 연산자)** 을 이용해 **체인형 파이프라인**을 구성해야 합니다.

##### 실습 시나리오
> 당신은 **AI Q&A 챗봇 개발자**입니다.  
> 사용자가 “기술 관련 질문”을 하면, 챗봇은 **요약 + 예시 + 비유 설명**을 포함한 답변을 하도록 만들어야 합니다.  
> (예: “REST API란?”, “클라우드 컴퓨팅이 뭐야?”, “RAG는 어떤 구조야?” 등)

즉, 단순한 정의를 넘어 “무엇인지 + 왜 중요한지 + 예시” 를 한 번에 설명하는 챗봇을 설계하세요.

**과제 요구사항**

1. **`ChatPromptTemplate`** 을 사용해 아래 구조의 프롬프트를 직접 설계하세요.
   - `system`: 챗봇의 역할 정의  
     (예: “너는 기술 개념을 쉬운 비유와 예시를 섞어서 설명하는 AI 선생님이야.”)
   - `user`: 사용자의 질문을 `{question}` 변수로 전달

2. **`ChatOpenAI`** 모델을 연결하세요.  
   - 모델: `"gpt-4o-mini"`  
   - `temperature=0.7` (조금 창의적인 답변 유도)

3. **`StrOutputParser`** 를 사용하여 결과를 문자열로 출력하세요.

4. **LCEL 문법**으로 체인을 완성하세요.  
   예:  
   ```python
   chain = prompt | llm | parser
   ```
5. 실행해보고 답변이 지나치게 딱딱하거나 비유나 예시가 부족하면 Prompt를 직접 수정하면서 성능을 조정해보세요.

**질문 답변 예시:**  

질문 : REST API란?  

답변 :   

① **정의**: REST API(Representational State Transfer Application Programming Interface)는 웹 상에서 서로 다른 프로그램이나 시스템이 데이터를 주고받을 수 있도록 하는 규칙과 방법을 정의한 인터페이스입니다. REST는 HTTP 프로토콜을 기반으로 하며, 자원(Resource)을 URI(Uniform Resource Identifier)로 식별하고, 다양한 HTTP 메서드(예: GET, POST, PUT, DELETE)를 사용하여 자원에 대한 작업을 수행합니다.

② **이유(중요성)**: REST API는 서로 다른 시스템 간의 통신을 쉽고 효율적으로 만들어줍니다. 이를 통해 개발자들은 복잡한 시스템을 간단하게 연결하고, 데이터를 주고받을 수 있습니다. REST API는 웹 기반 애플리케이션에서 표준으로 자리잡고 있어, 다양한 서비스를 통합하고 확장하는 데 매우 중요합니다.

③ **쉬운 예시**: REST API를 설명하기 위해 레스토랑을 비유로 들어볼 수 있습니다. 레스토랑의 메뉴판은 고객이 어떤 음식을 주문할 수 있는지를 보여주는 역할을 합니다. 고객(클라이언트)은 메뉴판(URI)을 보고 원하는 음식을 선택하고, 주문(HTTP 메서드)을 합니다. 주방(서버)은 고객의 주문을 받아 음식을 준비하고, 다시 고객에게 서빙합니다. 이 과정에서 메뉴판은 고객과 주방 간의 소통을 원활하게 해주는 역할을 하며, REST API는 시스템 간의 소통을 원활하게 해주는 역할을 합니다.


In [28]:
### 여기에 과제 요구 사항을 작성하세요
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


# 모델 선언
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7, max_tokens=200)

# 프롬프트 설계
# system : 챗봇의 역할 정의
# user : {question} 변수를 통해질문 전달
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "너는 기술 개념을 알기 쉽게 설명하는 AI 선생님이야. "
     "답변은 반드시 정의, 이유(중요성), 쉬운 예시를 포함해야 해"
     "필요하다면 비유를 사용해도 좋아"),

    # [예시 1] — 기술 중급자용 질문 예시1
    ("user", "프로세스랑 스레드는 뭐가 달라?"),
    ("assistant", "프로세스는 실행 중인 프로그램이고, 스레드는 그 안에서 일하는 작업 단위야."),
    ("assistant", "스레드를 쓰면 더 효율적으로 동시에 작업할 수 있어."),
    ("assistant", "브라우저 하나는 프로세스, 탭 처리나 입력 처리는 스레드야."),
    
    # [예시 2] — 기술 중급자용 질문 예시 2
    ("user", "API는 왜 규칙을 지켜야 해?"),
    ("assistant", "REST는 URL과 HTTP 메서드로 자원을 다루는 방식이야."),  
    ("assistant", "규칙이 있어야 협업과 유지보수가 쉬워."),  
    ("assistant", "GET /users는 조회, POST /users는 생성이야."),  

    # [새로운 요청] — 실행 시 전달될 질문 변수. 이제 모델이 같은 말투로 이어서 대답해야 함       
    ("user", "{question}")
])

# 출력 파서 (응답 텍스트만 추출)
parser = StrOutputParser()

# LCEL 파이프라인 구성 (Prompt → LLM → Parser)
chain = prompt | llm | parser

# 실행 — invoke()에 직접 새로운 질문 전달
result = chain.invoke({"question": "데이터베이스 정규화는 왜 해?"})

print(result)

**정의**: 데이터베이스 정규화는 데이터를 구조화하여 중복을 최소화하고 데이터 무결성을 높이는 과정을 말해. 주로 여러 단계로 나누어 진행되며, 각 단계에서 특정한 규칙을 따르지.

**이유 (중요성)**: 정규화를 통해 데이터의 중복을 줄이고, 데이터의 일관성을 유지할 수 있어. 예를 들어, 동일한 정보를 여러 곳에 저장하면 업데이트할 때 모든 곳을 수정해야 하므로 실수할 가능성이 커져. 이런 실수를 방지하고 데이터를 효율적으로 관리하기 위해 정규화가 필요해.

**쉬운 예시**: 상상해봐, 친구들과 함께 피자를 주문할 때 각자 좋아하는 토핑을 적어내는 상황이야. 만약 모든 친구가 "치즈, 올리브, 페퍼로니"라고 적는 대신, 같은 토핑이


<details>
<summary>정답 보기</summary>

```python 
from langchain_openai import ChatOpenAI
from langchain_core.prompts  import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 모델 선언
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7,  max_tokens=200)

# 프롬프트 설계
# system: 챗봇의 역할 정의
# user: {question} 변수를 통해 질문 전달
prompt = ChatPromptTemplate.from_messages([
    ("system", 
     "너는 기술 개념을 알기 쉽게 설명하는 AI 선생님이야. "
     "답변은 반드시 ① 정의 ② 이유(중요성) ③ 쉬운 예시를 포함해야 해. "
     "필요하다면 비유를 사용해도 좋아."),
    ("user", "{question}")
])

# 출력 파서
parser = StrOutputParser()

# LCEL 체인 구성
chain = prompt | llm | parser

# 실행 테스트
question = "REST API란?"
answer = chain.invoke({"question": question})

print(f"🧠 질문: {question}\n")
print(f"💬 답변:\n{answer}")
```
</details>

##  5. 대화형 챗봇 (메모리)

### 🔹 Conversation Memory란?
> **Conversation Memory(대화 메모리)** 는 LLM이 **이전 대화 내용을 기억하고 이어서 대답**할 수 있도록 하는 기능입니다.

일반적인 LLM은 한 번의 요청(prompt)만 처리하고 끝나기 때문에,  
사용자가 “아까 말한 여행지 일정 다시 알려줘.”처럼 과거 대화를 참조하면 맥락을 잃어버립니다.  
→ 이때 필요한 것이 바로 **Memory(기억 기능)** 입니다.

### 🔹 왜 Memory가 필요할까?

| 상황 | Memory 없을 때 | Memory 있을 때 |
|------|----------------|----------------|
| 사용자: “부산 여행지 추천해줘.”<br>→ 모델이 답변함 | 다음 질문: “그럼 거기 근처 맛집은?”<br>→ 모델이 “어디 여행 말씀인가요?” | “부산 근처엔 회센터가 많고… 홍합탕집이 유명해요!” |
| 사용자: “어제 추천해준 책 제목 다시 말해줘.” | “어떤 책을 말씀하시는 건가요?” | “어제 말씀드린 건 『데미안』이에요.” |

👉 메모리를 활용하면,  
모델이 “과거 대화의 맥락(Context)”을 유지한 채로  
**자연스러운 멀티턴(Multi-turn) 대화형 챗봇**을 만들 수 있습니다.

### 🔹 LangChain에서의 Memory 개념

LangChain은 다양한 형태의 “기억 클래스(memory classes)”를 제공합니다.  
대표적으로 아래 세 가지를 이해하면 충분합니다 👇

| Memory 클래스 | 설명 | 특징 |
|----------------|------|------|
| **`ConversationBufferMemory`** | 단순히 대화 전체를 계속 저장 | 가장 기본적이고 직관적 |
| **`ConversationBufferWindowMemory`** | 최근 N개의 대화만 저장 | “단기 기억” 형태로, 긴 대화를 효율적으로 유지 |
| **`ConversationSummaryMemory`** | 이전 대화를 LLM이 요약해서 저장 | 긴 대화를 “핵심 요약본”으로 관리, 장기 기억에 적합 |

### 🔹 프롬프트에 직접 넣는 방식 vs Memory 클래스 사용 비교

| 비교 항목 | 프롬프트에 직접 삽입 | Memory 클래스 사용 |
|------------|----------------------|---------------------|
| 코드 복잡도 | ❌ 대화마다 프롬프트를 새로 생성해야 함 | ✅ LangChain이 자동으로 이전 대화 삽입 |
| 확장성 | ❌ 대화가 길어지면 토큰 초과 문제 발생 | ✅ 오래된 대화는 요약/제한 가능 |
| 유지보수 | ⚠️ 과거 대화 삽입 위치만 교체하면 되지만, 요약·토큰 제어를 직접 구현해야 함 | ✅ 메모리 객체만 관리하면 됨 |
| 현실성 | ❌ “일회용 챗봇”에만 적합 | ✅ “지속형 대화형 챗봇” 구현 가능 |

즉, **Memory는 LLM이 “대화 히스토리를 스스로 관리”하도록 하는 스마트한 도우미 클래스**입니다.

### 🔹 Memory 작동 원리
1. 사용자가 LLM에 **질문(입력)** 을 보냅니다.  

2. LLM이 **답변**을 생성하면, 이 대화 내용이 자동으로 **Memory에 저장**됩니다.  
3. 사용자가 다음 질문을 입력하면,  
   LangChain이 **이전 대화 기록을 자동으로 프롬프트에 포함시켜** LLM에 전달합니다.  
4. LLM은 이 정보를 바탕으로 **맥락을 이해하고 연속적인 대화**를 생성합니다.

→ 즉, 사용자는 별도로 과거 대화를 넣을 필요 없이, LangChain이 **“대화 기록 → 프롬프트 삽입 → LLM 호출”**  
과정을 자동으로 처리합니다.

### 코드 예시 1. 기본 Buffer Memory (대화 전체 저장)

In [32]:
### 코드 예시 ① 기본 Buffer Memory
from langchain_openai import ChatOpenAI
from langchain_classic.memory import ConversationBufferMemory
from langchain_core.prompts  import ChatPromptTemplate, MessagesPlaceholder

# 모델 선언
llm = ChatOpenAI(model="gpt-4o-mini")

# 메모리 객체 생성 (대화 전체를 버퍼 형태로 저장)
memory = ConversationBufferMemory(return_messages=True)

# 프롬프트 템플릿
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 여행 전문가야. 사용자의 질문에 친절하게 답해줘."),
    MessagesPlaceholder(variable_name="history"),  # 이전 대화 삽입 위치
    ("human", "{input}")                           # 사용자 질문
]) 

# LCEL 체인 구성
chain = prompt | llm 

# 5️⃣ 연속 대화 시뮬레이션
inputs = ["부산 여행지 추천해줘.", "그럼 그 근처 맛집은 어디야?","그중에서 중국집을 알려줘"]
#inputs = ["울산 여행지 추천해줘.", "그럼 그 근처 맛집은 어디야?"]

for user_input in inputs:
    # 메모리 불러오기
    history = memory.load_memory_variables({})["history"]
    # 실행
    result = chain.invoke({"history": history, "input": user_input})
    # 결과 출력
    print(f"\n사용자: {user_input}\n 응답: {result.content}")
    # 메모리에 저장
    memory.save_context({"input": user_input}, {"output": result.content})


사용자: 부산 여행지 추천해줘.
 응답: 부산은 다채로운 매력을 가진 도시로, 여행지로 매우 인기가 높습니다. 아래는 부산에서 꼭 가봐야 할 여행지 몇 곳을 추천해드릴게요.

1. **해운대 해수욕장**: 부산에서 가장 유명한 해변으로, 여름철에 많은 관광객이 찾습니다. 주변에는 다양한 카페와 레스토랑이 있어 산책하면서 먹거리를 즐기기 좋습니다.

2. **광안리 해수욕장**: 해운대보다 조금 조용한 분위기를 원하는 분들에게 적합합니다. 특히 밤에 광안대교의 야경이 아름다워 로맨틱한 저녁을 즐길 수 있습니다.

3. **부산 타워**: 부산의 전경을 한눈에 볼 수 있는 전망대입니다. 특히 해가 지고 나서의 야경이 매력적입니다.

4. **자갈치 시장**: 부산의 대표적인 수산물 시장으로, 신선한 해산물을 맛볼 수 있습니다. 현지인과 관광객 모두에게 인기가 많습니다.

5. **감천문화마을**: 아기자기한 색깔의 집들과 예술 작품들이 어우러진 마을로, 사진 찍기 좋은 장소입니다. 마을을 걸으며 다양한 예술작품을 즐길 수 있습니다.

6. **부산 영화의 전당**: 부산국제영화제가 열리는 장소로, 영화 관련 이벤트와 전시가 자주 열립니다. 현대적인 건축물 자체도 볼 가치가 있습니다.

7. **온천장**: 전통적인 찜질방 체험을 할 수 있는 곳으로, 피로를 풀고 편안한 시간을 갖기에 좋습니다.

8. **태종대**: 아름다운 해안 절경을 감상할 수 있는 곳으로, 다채로운 산책로가 있어 자연을 즐기며 여유롭게 걷기 좋습니다.

부산은 다양한 매력을 가진 도시이기 때문에 여행 일정에 맞춰 방문하시면 좋을 것 같습니다. 즐거운 여행 되세요!

사용자: 그럼 그 근처 맛집은 어디야?
 응답: 부산은 맛있는 음식의 천국이죠! 각 여행지 근처에 있는 맛집들을 추천해드릴게요.

1. **해운대 해수욕장 근처**
   - **동백섬**: 해운대에서 가까운 위치의 전복죽 맛집입니다. 신선한 해산물을 사용한 전복죽이 유명합니다.
   - **해운대 암소갈비**: 양념갈비와 찜갈비

모델은 앞선 “부산 여행” 대화를 기억하고 자연스럽게 이어서 대답합니다.

In [33]:
memory.load_memory_variables({})['history']

[HumanMessage(content='부산 여행지 추천해줘.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='부산은 다채로운 매력을 가진 도시로, 여행지로 매우 인기가 높습니다. 아래는 부산에서 꼭 가봐야 할 여행지 몇 곳을 추천해드릴게요.\n\n1. **해운대 해수욕장**: 부산에서 가장 유명한 해변으로, 여름철에 많은 관광객이 찾습니다. 주변에는 다양한 카페와 레스토랑이 있어 산책하면서 먹거리를 즐기기 좋습니다.\n\n2. **광안리 해수욕장**: 해운대보다 조금 조용한 분위기를 원하는 분들에게 적합합니다. 특히 밤에 광안대교의 야경이 아름다워 로맨틱한 저녁을 즐길 수 있습니다.\n\n3. **부산 타워**: 부산의 전경을 한눈에 볼 수 있는 전망대입니다. 특히 해가 지고 나서의 야경이 매력적입니다.\n\n4. **자갈치 시장**: 부산의 대표적인 수산물 시장으로, 신선한 해산물을 맛볼 수 있습니다. 현지인과 관광객 모두에게 인기가 많습니다.\n\n5. **감천문화마을**: 아기자기한 색깔의 집들과 예술 작품들이 어우러진 마을로, 사진 찍기 좋은 장소입니다. 마을을 걸으며 다양한 예술작품을 즐길 수 있습니다.\n\n6. **부산 영화의 전당**: 부산국제영화제가 열리는 장소로, 영화 관련 이벤트와 전시가 자주 열립니다. 현대적인 건축물 자체도 볼 가치가 있습니다.\n\n7. **온천장**: 전통적인 찜질방 체험을 할 수 있는 곳으로, 피로를 풀고 편안한 시간을 갖기에 좋습니다.\n\n8. **태종대**: 아름다운 해안 절경을 감상할 수 있는 곳으로, 다채로운 산책로가 있어 자연을 즐기며 여유롭게 걷기 좋습니다.\n\n부산은 다양한 매력을 가진 도시이기 때문에 여행 일정에 맞춰 방문하시면 좋을 것 같습니다. 즐거운 여행 되세요!', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 Hu

### 코드 예시 2. 최근 대화만 유지 (Window Memory)

In [34]:
### 코드 예시 ① 기본 Buffer Memory
from langchain_openai import ChatOpenAI
from langchain_classic.memory import ConversationBufferWindowMemory
from langchain_core.prompts  import ChatPromptTemplate, MessagesPlaceholder

# 모델 선언
llm = ChatOpenAI(model="gpt-4o-mini")

# 메모리 객체 생성 (대화 전체를 버퍼 형태로 저장)
memory = ConversationBufferWindowMemory(k=2, return_messages=True)

# 프롬프트 템플릿
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 여행 전문가야. 사용자의 질문에 친절하게 답해줘."),
    MessagesPlaceholder(variable_name="history"),  # 이전 대화 삽입 위치
    ("human", "{input}")                           # 사용자 질문
]) 

# LCEL 체인 구성
chain = prompt | llm 

# 5️⃣ 연속 대화 시뮬레이션
inputs = ["부산 여행지 추천해줘.", "대한 민국의 수도는 어디야?", "서울의 인구는 몇명이야?", "내가 아까 추천해달라고 한 어행지는 어디야?" ]

for user_input in inputs:
    # 메모리 불러오기
    history = memory.load_memory_variables({})["history"]
    # 실행
    result = chain.invoke({"history": history, "input": user_input})
    # 결과 출력
    print(f"\n사용자: {user_input}\n 응답: {result.content}")
    # 메모리에 저장
    memory.save_context({"input": user_input}, {"output": result.content})


C:\Users\HPE\AppData\Local\Temp\ipykernel_11332\2340361679.py:10: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=2, return_messages=True)



사용자: 부산 여행지 추천해줘.
 응답: 부산은 아름다운 해변과 다양한 문화, 맛있는 음식으로 유명한 도시입니다. 여기 몇 가지 추천 여행지를 소개해드릴게요:

1. **해운대 해수욕장**: 부산에서 가장 유명한 해변으로, 여름철에는 많은 관광객이 찾습니다. 해변가에 있는 카페와 레스토랑, 그리고 동백섬도 함께 즐길 수 있습니다.

2. **광안리 해수욕장**: 광안대교의 아름다운 경관을 감상할 수 있는 곳으로, 저녁에는 야경이 특히 아름답습니다. 수많은 음식점과 카페가 있어 맛있는 부산 음식을 즐길 수 있습니다.

3. **감천문화마을**: 독특한 색깔의 집들이 모여 있는 마을로, 아기자기한 골목길과 예술작품들이 많아 사진찍기 좋은 장소입니다. 

4. **부산타워**: 용두산공원에 위치한 부산타워에서는 부산 시내와 바다의 전망을 한눈에 감상할 수 있습니다. 특히, 석양 때 방문하면 더욱 아름답습니다.

5. **자갈치 시장**: 부산의 대표적인 수산시장으로, 신선한 해산물을 직접 구매하거나 먹을 수 있습니다. 현지 음식과 문화를 체험할 수 있는 좋은 장소입니다.

6. **영도 해양공원**: 바다를 따라 조성된 공원으로, 산책하기 좋으며 해상 케이블카를 타고 멋진 전망을 감상할 수 있습니다.

7. **서면**: 부산의 핫플레이스 중 하나로, 쇼핑과 맛집이 많은 지역입니다. 특히, 밤에는 활기 넘치는 분위기를 즐길 수 있습니다.

이 외에도 많은 매력적인 장소가 있으니, 부산 여행을 즐기시길 바랍니다! 필요하신 다른 정보가 있다면 얼마든지 물어보세요.

사용자: 대한 민국의 수도는 어디야?
 응답: 대한민국의 수도는 서울입니다. 서울은 정치, 경제, 문화의 중심지로, 다양한 역사적 명소와 현대적인 시설이 어우러져 있습니다. 궁궐, 박물관, 공원 등 볼거리가 많아 여행객들에게도 인기 있는 도시입니다. 추가적인 정보가 필요하시다면 언제든지 질문해 주세요!

사용자: 서울의 인구는 몇명이야?
 응답: 2023년 기준으로 서울의 인구는 약 9백만 명 정도입니다.

이 방식은 단기 기억처럼 작동합니다.  
오래된 대화는 자동으로 지워져 토큰 낭비를 줄이고 효율성을 높입니다.

### 코드 예시 3. 이전 대화를 요약해서 저장 (ConversationSummaryMemory)

In [39]:
from langchain_openai import ChatOpenAI
from langchain_classic.memory import ConversationSummaryMemory
from langchain_core.prompts  import ChatPromptTemplate, MessagesPlaceholder

# 모델 선언
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

# 요약형 메모리 생성 — LLM이 과거 대화를 자동 요약하여 저장
memory = ConversationSummaryMemory(llm=llm, return_messages=True)

# 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 여행 플래너야. 사용자의 요구에 따라 가족 여행 일정을 제안해줘."),
    MessagesPlaceholder(variable_name="history"),  # 요약된 과거 대화 자동 삽입
    ("human", "{input}")                           # 현재 질문
])

print(memory.buffer)

# LCEL 체인 구성
chain = prompt | llm

# 연속 대화 시뮬레이션
inputs = [
    "이번 주말에 가족 여행지 추천해줘.",
    "지난번에 추천한 곳 중에 아이들이 놀기 좋은 곳은 어디였지?",
    "그럼 거기 일정표를 하루만 짜줘."
]

for user_input in inputs:
    history = memory.load_memory_variables({})["history"]
    result = chain.invoke({"history": history, "input": user_input})
    print(f"\n👤 사용자: {user_input}\n🤖 응답: {result.content}")
    memory.save_context({"input": user_input}, {"output": result.content})
    
print(memory.buffer)



👤 사용자: 이번 주말에 가족 여행지 추천해줘.
🤖 응답: 이번 주말 가족 여행을 위한 몇 가지 추천지를 제안해드릴게요. 가족 구성원들의 나이와 취향에 따라 선택하시면 좋을 것 같습니다!

### 1. 제주도
- **추천 활동**: 한라산 등반, 성산 일출봉, 만장굴 탐험, 우도 자전거 타기
- **추천 음식**: 흑돼지, 갈치 조림, 제주 감귤
- **추천 숙소**: 리조트 또는 펜션에서의 가족 단위 숙박

### 2. 강릉
- **추천 활동**: 경포대 해수욕장, 주문진 수산시장 방문, 오죽헌 탐방
- **추천 음식**: 강릉 커피, 회, 메밀국수
- **추천 숙소**: 바다 전망이 있는 호텔 또는 게스트하우스

### 3. 남이섬
- **추천 활동**: 자전거 타기, 나무길 산책, 수목원 탐방
- **추천 음식**: 남이섬 특산물인 닭갈비
- **추천 숙소**: 섬 내 숙소나 근처 펜션

### 4. 경주
- **추천 활동**: 불국사, 석굴암, 경주월드 테마파크
- **추천 음식**: 경주 특산물인 황남빵
- **추천 숙소**: 전통 한옥이나 호텔

### 5. 부산
- **추천 활동**: 해운대 해수욕장, 감천문화마을, 자갈치 시장
- **추천 음식**: 회, 밀면, 씨앗호떡
- **추천 숙소**: 해변 근처의 리조트나 호텔

각 지역마다 다양한 볼거리와 먹거리가 있으니 가족 구성원들의 취향에 맞춰 선택해보세요! 안전하고 즐거운 여행 되시길 바랍니다!

👤 사용자: 지난번에 추천한 곳 중에 아이들이 놀기 좋은 곳은 어디였지?
🤖 응답: 아이들이 놀기 좋은 장소로는 다음과 같은 곳들이 있습니다:

1. **제주도**: 제주도는 다양한 놀이공원과 자연 체험 활동이 많아 아이들이 즐길 수 있는 최적의 장소입니다. 성산 일출봉, 한라산 등 자연을 즐길 수 있고, 제주 아쿠아플라넷, 제주 테디벨리 등 아이들이 좋아할 만한 시설도 많습니다. 제주 해산물과 흑돼지도 잊지 말고 맛보세요.

2. **강릉**: 강릉은 해변과 자연이 어우러져 있어 아이들과 함께해도

In [40]:
# 현재까지의 요약본 확인
print(memory.buffer)

The human asks the AI to create a one-day itinerary for a family trip to Jeju Island. The AI provides a detailed schedule that includes morning activities like a beach walk near Hallim Park followed by a visit to either the Lotte World Aquarium or Eco Land theme park, with breakfast suggestions nearby. For lunch, the AI recommends a famous black pork BBQ restaurant. In the afternoon, the itinerary includes a hike at Seongsan Ilchulbong for scenic views and a boat trip to Udo Island for beach fun and a bike tour. Dinner suggestions feature a seafood restaurant, and the evening ends with a stroll to enjoy Jeju's night views or relaxing at the accommodation. The AI emphasizes that the itinerary is designed with family-friendly activities in mind to ensure a fun trip for children.


이 방식은 LLM이 이전 대화를 스스로 요약하여 “장기 기억” 처럼 보관합니다.  
수백 문장의 대화도 핵심 내용만 남기기 때문에 효율적입니다.

**핵심 정리**
| Memory 타입 | 특징 | 사용 목적 |
|--------------|--------|-------------|
| **BufferMemory** | 모든 대화를 그대로 저장 | 짧은 대화, 디버깅용 |
| **WindowMemory** | 최근 N개의 대화만 유지 | 실시간 채팅, 효율성 |
| **SummaryMemory** | 이전 대화를 요약해서 유지 | 장기 기억, 맥락 유지형 챗봇 |

🔹 실습 : 아래 조건을 만족하는 “나만의 대화형 여행 상담 챗봇”을 만들어보세요.

당신은 AI 여행 비서 트래블GPT 입니다.  
고객이 여러 도시를 순서대로 여행하면서 맞춤 일정·숙소·음식을 요청할 때,  
이전 대화 내용을 기억하며 연결된 제안을 해주는 지능형 여행 어시스턴트를 구현하세요.  

1. ChatOpenAI(model="gpt-4o-mini") 사용
2. ConversationSummaryMemory로 장기 대화 기억 구현
3. 답변에 반드시 "이전 여행 내용을 바탕으로 추천드리면..." 이라는 문구 포함
  - 예시 : 이전 여행 내용을 바탕으로 추천드리면, 여수에서는 해상케이블카와 낭만포차거리를 꼭 가보세요.
4. 대화 시나리오:
  - 사용자가 “부산 → 여수 → 강릉” 순으로 도시를 이동
  - 챗봇은 이전 도시에서 한 활동을 기억하고 “연결된 여행 루트”나 “테마별 추천(가족/커플/힐링)”을 제안할 것

예시 시나리오
```bash
사용자: 이번 주말엔 부산 갈 건데 가족 여행지 좀 추천해줘.
AI: 부산의 해운대, 아쿠아리움이 가족 단위로 인기예요!

사용자: 이번엔 여수로 가볼까?
AI: 이전 여행 내용을 바탕으로 추천드리면, 부산의 해변 감성에 이어 여수에서는 바다 전망 케이블카와 낭만포차를 즐기세요.

사용자: 그럼 마지막은 강릉이 좋을까?
AI: 이전 여행 내용을 바탕으로 추천드리면, 강릉에서는 여수보다 조용한 힐링 카페 거리와 바다 일출 코스를 권합니다.
```

In [41]:
from langchain_openai import ChatOpenAI
from langchain_classic.memory import ConversationSummaryMemory
from langchain_core.prompts  import ChatPromptTemplate, MessagesPlaceholder

# 모델 선언
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

# 메모리 생성 — 이전 대화를 요약하며 장기 기억
memory = ConversationSummaryMemory(llm=llm, return_messages=True)

# 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "너는 여행 비서 트래블GPT야. "
     "사용자의 여행 루트를 기억하고, 이전 여행 내용을 바탕으로 다음 도시를 추천해줘. "
     "답변에는 반드시 '이전 여행 내용을 바탕으로 추천드리면,' 이라는 문구를 포함해야 해."),
    MessagesPlaceholder(variable_name="history"),  # 과거 대화 자동 삽입
    ("human", "{input}")                            # 현재 사용자 입력
])

# LCEL 체인 구성
chain = prompt | llm

# 대화 시나리오
inputs = [
    "이번 주말엔 부산 갈 건데 가족 여행지 좀 추천해줘.",
    "이번엔 여수로 가볼까?",
    "그럼 마지막은 강릉이 좋을까?"
]

# 연속 대화 시뮬레이션
for user_input in inputs:
    history = memory.load_memory_variables({})["history"]
    result = chain.invoke({"history": history, "input": user_input})
    print(f"\n 사용자: {user_input}\n 트래블GPT: {result.content}")
    memory.save_context({"input": user_input}, {"output": result.content})

# 대화 요약 확인 (선택)
print("\n 요약된 Memory Buffer:")
print(memory.buffer)


 사용자: 이번 주말엔 부산 갈 건데 가족 여행지 좀 추천해줘.
 트래블GPT: 이전 여행 내용을 바탕으로 추천드리면, 부산에서 가족과 함께 즐길 수 있는 몇 가지 여행지를 소개해드릴게요.

1. **해운대 해수욕장**: 부산의 대표적인 해수욕장으로, 모래사장에서 여유로운 시간을 보낼 수 있습니다. 근처에 있는 동백섬도 산책하기 좋습니다.

2. **부산 아쿠아리움**: 해운대 해수욕장 근처에 위치하여 아이들과 함께 다양한 해양 생물을 관찰할 수 있는 좋은 장소입니다.

3. **감천문화마을**: 알록달록한 집들이 있는 이곳은 가족과 함께 사진 찍기 좋은 포토 스팟입니다. 마을을 산책하며 예술 작품들도 감상할 수 있습니다.

4. **부산타워**: 용두산 공원에 위치한 부산타워에서 부산 전경을 한눈에 내려다보며 멋진 사진을 남길 수 있습니다.

5. **광안리 해변**: 저녁에는 광안대교의 야경을 즐길 수 있는 아름다운 해변입니다. 가족과 함께 산책하며 시간을 보내기에 좋습니다.

부산에서 즐거운 시간 보내세요!

 사용자: 이번엔 여수로 가볼까?
 트래블GPT: 이전 여행 내용을 바탕으로 추천드리면, 여수에서 가족과 함께 즐길 수 있는 멋진 장소들이 많습니다. 

1. **여수 해상 케이블카**: 바다 위를 가로지르는 케이블카를 타고 아름다운 경치를 감상할 수 있습니다. 아이들과 함께 즐기기 좋은 액티비티입니다.

2. **오동도**: 조용한 산책로와 아름다운 풍경이 있는 오동도에서 자연을 만끽해보세요. 특히, 봄에는 동백꽃이 아름답습니다.

3. **여수 수산시장**: 신선한 해산물을 맛볼 수 있는 수산시장에서 다양한 음식을 즐겨보세요. 가족과 함께 시장 구경도 재미있습니다.

4. **돌산대교**: 돌산대교에서의 일몰은 정말 아름답습니다. 사진 찍기에도 좋은 장소입니다.

5. **향일암**: 바다를 바라보는 절경을 감상할 수 있는 장소로, 가족과 함께 평화로운 시간을 보낼 수 있습니다.

여수에서 즐거운 시간 보내시길 바랍니다!

 사용자: 그럼 마지